# IP Notebook for ADS Fall 2015 Networks Project on US State Tax Rates
## Jiheng, Linda, Juan

In [7]:
# add necessary libraries
import networkx as nx #library supporting networks
import matplotlib.pyplot as plt #plotting
import pandas as pd
import geopandas as gpd
import numpy as np
import scipy.stats as stat
from scipy import optimize
import pysal as ps
#from zipfile import ZipFile
#from StringIO import StringIO
# make sure plots are embedded into the notebook
%pylab inline 
import statsmodels.formula.api as smf

Populating the interactive namespace from numpy and matplotlib


## 1. Data Loading

In [8]:
# load state gdp data (2014)
gdp = pd.read_csv('bea_gdp_by_state_in_millions.csv', header = 0, names=['fips', 'state', 'gdp'], index_col=['state'])
print gdp.head()
print gdp.columns
print len(gdp)

            fips      gdp
state                    
Alabama     1000   199440
Alaska      2000    57080
Arizona     4000   284156
Arkansas    5000   121395
California  6000  2311616
Index([u'fips', u'gdp'], dtype='object')
51


In [9]:
# load state capitals
caps = pd.read_csv('Capitals.csv', header = 0, index_col=['state'])
print caps.head()
print caps.columns
print len(caps)

            id abbrev      capital   latitude   longitude  population
state                                                                
Alabama      1     AL   Montgomery  32.380120  -86.300629      205764
Alaska       2     AK       Juneau  58.299740 -134.406794       31275
Arizona      4     AZ      Phoenix  33.448260 -112.075774     1445632
Arkansas     5     AR  Little Rock  34.748655  -92.274494      193524
California   6     CA   Sacramento  38.579065 -121.491014      466488
Index([u'id', u'abbrev', u'capital', u'latitude', u'longitude', u'population'], dtype='object')
50


In [10]:
# load state population centers
popcenter = pd.read_csv('CenPop2010_Mean_ST.txt', index_col=['STNAME'])
print popcenter.head()
print popcenter.columns

            STATEFP  POPULATION   LATITUDE   LONGITUDE
STNAME                                                
Alabama           1     4779736  33.008097  -86.756826
Alaska            2      710231  61.399882 -148.873973
Arizona           4     6392017  33.368266 -111.864310
Arkansas          5     2915918  35.142580  -92.655243
California        6    37253956  35.463595 -119.325359
Index([u'STATEFP', u'POPULATION', u'LATITUDE', u'LONGITUDE'], dtype='object')


In [11]:
# load state tax rates (all types)
tax = pd.read_excel('Taxes rates by state.xlsx', index_col=['State'])
print len(tax)

# clean col names to make easier to work with
tax.columns = ['State_Sales', 'Avg_Local_Sales', 'Combined_Sales', 'Max_Local_Sales'
               , 'Property', 'Income_Low', 'Income_High', 'Mature_Firm_HQ', 'New_Firm_HQ']
print tax.columns
#print tax.head()

# clean index names
tax.index =  [state.replace("\"", "").strip() for state in tax.index]
#print tax.index

# convert percentages to floats
tax['New_Firm_HQ'] = tax['New_Firm_HQ'].replace('%','',regex=True).astype('float')/100
tax['Mature_Firm_HQ'] = tax['Mature_Firm_HQ'].replace('%','',regex=True).astype('float')/100

tax = tax.fillna(0)

51
Index([u'State_Sales', u'Avg_Local_Sales', u'Combined_Sales',
       u'Max_Local_Sales', u'Property', u'Income_Low', u'Income_High',
       u'Mature_Firm_HQ', u'New_Firm_HQ'],
      dtype='object')


In [12]:
# What makes intuitive sense

'''
NONGRAVITY MODELS
Combined_Sales: highest spatial auto-correlation based on rook/queen
    you can only travel so far to buy stuff
Property: low spatial auto-corr (rook > queen marginally)
    since tied to land

POTENTIAL FOR GRAVITY MODEL
Income-Low vs. Income-High: lower for low income than for high income since richer people are more mobile
    also, look at gravity model here
Mature_Firm_HQ: high spatial auto-correlation
    also gravity model
New_Firm_HQ: even higher spatial auto-corr than mature
    new firms are more mobile than mature firms)
'''

'\nNONGRAVITY MODELS\nCombined_Sales: highest spatial auto-correlation based on rook/queen\n    you can only travel so far to buy stuff\nProperty: low spatial auto-corr (rook > queen marginally)\n    since tied to land\n\nPOTENTIAL FOR GRAVITY MODEL\nIncome-Low vs. Income-High: lower for low income than for high income since richer people are more mobile\n    also, look at gravity model here\nMature_Firm_HQ: high spatial auto-correlation\n    also gravity model\nNew_Firm_HQ: even higher spatial auto-corr than mature\n    new firms are more mobile than mature firms)\n'

In [13]:
# This is only for the shapes that will be used by PySAL to 
# build the spatial weights matrix
data = gpd.read_file('cb_2014_us_state_5m/cb_2014_us_state_5m.shp')
psGeom = ps.open('cb_2014_us_state_5m/cb_2014_us_state_5m.shp', 'r')

print data.columns

# clean state names
data['NAME'] = [statename.strip() for statename in data['NAME']]

Index([u'AFFGEOID',    u'ALAND',   u'AWATER',    u'GEOID',     u'LSAD',
           u'NAME',  u'STATEFP',  u'STATENS',   u'STUSPS', u'geometry'],
      dtype='object')


## 2. Build spatial weight matrices

### 2.1. Rook 

In [14]:
# We are building the spatial weight matrix and using the 
# state names as IDs of the matrix.

R = ps.buildContiguity(psGeom, criterion='rook', ids=data['NAME'].values.tolist())
R.transform = 'R' # normalize

Island ids:  [u'Puerto Rico', u'Commonwealth of the Northern Mariana Islands', u'Alaska', u'Hawaii', u'United States Virgin Islands', u'American Samoa', u'Guam']


In [15]:
#for (loc, neighbors) in R:
 #   print loc, neighbors

In [16]:
print type(R)

<class 'pysal.weights.weights.W'>


### 2.2. Queen 

In [17]:
# We are building the spatial weight matrix and using the 
# state names as IDs of the matrix. Noted that we
# running a 'queen', shared vertices, neighborhood test.

Q = ps.buildContiguity(psGeom, criterion='queen', ids=data['NAME'].values.tolist())
Q.transform = 'R' # normalize

Island ids:  [u'Puerto Rico', u'Commonwealth of the Northern Mariana Islands', u'Alaska', u'Hawaii', u'United States Virgin Islands', u'American Samoa', u'Guam']


In [39]:
# 1. get data where tax rate is provided
print 'Number of States with Data: ', len(tax)
print
ids = tax.index.values.tolist()
#print ids # list of states with this tax rate provided

# 3. subset queen spatial weights matrix to only those states
Q_Sales = ps.w_subset(Q, ids)
Q_Sales.transform = 'R' # normalize
#print Q_Property.id_order

# 4. get and normalize tax rate values (dependent variable)
Y = tax['State_Sales'].values
Y = (Y-Y.mean())/Y.std() # <<<---- normalization

# 5. calculate Moran's statistics for spatial auto-corr. with ROOK and QUEEN
mi_Q = ps.Moran(Y, Q_Sales)
print
print 'Queen: ', mi_Q.I, mi_Q.p_sim

Number of States with Data:  51

Island ids:  [u'Alaska', u'Hawaii']

Queen:  -0.148484732177 0.084


### 2.3. Distance-weighted by population centers

In [21]:
# get latlong points
points = [(popcenter['LATITUDE'][i], popcenter['LONGITUDE'][i]) for i in popcenter.index.values.tolist()]

In [22]:
# get dict. keys for state names
ids = tax.index.values.tolist()

In [23]:
# custom latlong dist calc function
from math import sin, cos, sqrt, atan2, radians
def geodist(lon1,lat1,lon2,lat2):
    lat1 = radians(lat1)
    lon1 = radians(lon1)  
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    R = 6373.0
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

In [24]:
# define custom spatial weight matrix by first defining all states as neighbors
neighbors = {}
for state in ids:
    neighbors[state] = ids    

In [25]:
# define weights
weights = {}

#count the states
i = 0

for state in ids:
    result=[]
    for a in xrange(0, 51):
        if i == a: # self-weight = 0
            b = 0
        else:
            b = 1.0/ geodist(points[i][0],points[i][1],points[a][0],points[a][1])
        result.append(b)
    weights[state] = result
    i += 1

In [26]:
# build matrix
w = ps.W(neighbors, weights)

In [27]:
# normalize
w.transform = 'R'

In [28]:
# 1. subset distance weighted spatial weights matrix to only those states
# this step is actually not nec., from legacy formatting
D_Income = ps.w_subset(w, ids)
D_Income.transform = 'R' # normalize again

# 2. get and normalize tax rate values (dependent variable)
Y = tax['Income_High'].values
Y = (Y-Y.mean())/Y.std() # <<<---- normalization

# 3. calculate Moran's statistics for spatial auto-corr. with DISTANCE
mi_D = ps.Moran(Y, D_Income)
print
print 'Distance: ', mi_D.I, mi_D.p_sim


Distance:  -1.0573733028e-34 0.303


In [29]:
# 1. subset distance weighted spatial weights matrix to only those states
# this step is actually not nec., from legacy formatting
D_Income = ps.w_subset(w, ids)
D_Income.transform = 'R' # normalize again

# 2. get and normalize tax rate values (dependent variable)
Y = tax['Income_Low'].values
Y = (Y-Y.mean())/Y.std() # <<<---- normalization

# 3. calculate Moran's statistics for spatial auto-corr. with DISTANCE
mi_D = ps.Moran(Y, D_Income)
print
print 'Distance: ', mi_D.I, mi_D.p_sim


Distance:  0.0 0.083


### 2.4. Gravity model (with local normalization)

In [56]:
# buid spatial weight matrix again
weightsHQ = {}

# count the states
i = 0

for state in ids:
    result=[]
    for a in xrange(0, 51):
        dest = ids[a]
        if i == a:
            b = 0
        else:
            b = 1.0 * gdp['gdp'][dest]/ geodist(points[i][0],points[i][1],points[a][0],points[a][1]) 
        result.append(b)
    weightsHQ[state] = result
    i += 1

In [57]:
g = ps.W(neighbors, weightsHQ)

In [58]:
g.transform = "R"

In [59]:
# 1. subset spatial weights matrix to only those states
G_HQTax = ps.w_subset(g, ids)
G_HQTax.transform = 'R' # normalize

# 2. get and normalize tax rate values (dependent variable)
Y = tax['New_Firm_HQ'].values
Y = (Y-Y.mean())/Y.std() # <<<---- normalization

# 3. calculate Moran's statistics for spatial auto-corr. with GRAVITY (MASS AND DISTANCE)
mi_G = ps.Moran(Y, G_HQTax)
print
print 'HQ Tax Gravity: ', mi_G.I, mi_G.p_sim


HQ Tax Gravity:  -3.02106657943e-34 0.028


In [60]:
# 1. subset spatial weights matrix to only those states
G_HQTax = ps.w_subset(g, ids)
G_HQTax.transform = 'R' # normalize

# 2. get and normalize tax rate values (dependent variable)
Y = tax['Mature_Firm_HQ'].values
Y = (Y-Y.mean())/Y.std() # <<<---- normalization

# 3. calculate Moran's statistics for spatial auto-corr. with GRAVITY (MASS AND DISTANCE)
mi_G = ps.Moran(Y, G_HQTax)
print
print 'HQ Tax Gravity: ', mi_G.I, mi_G.p_sim


HQ Tax Gravity:  2.41685326354e-34 0.157
